In [1]:
#!pip uninstall huggingface_hub -y && pip install git+https://github.com/huggingface/huggingface_hub
#!pip install jiwer

In [2]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, Dataset
from tqdm import tqdm
import datasets

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True,nb_workers=8)
tqdm.pandas()

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer, 
    TrainingArguments, Trainer, DataCollatorForSeq2Seq,AutoTokenizer,FlaxT5ForConditionalGeneration,AutoModelForSeq2SeqLM
  )


In [4]:
model_name = 'flax-community/bengali-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/944M [00:00<?, ?B/s]

In [5]:
train_data = datasets.Dataset.load_from_disk('/kaggle/input/t5-asr-corrector-masked-second-attempt-data/train')
test_data  = datasets.Dataset.load_from_disk('/kaggle/input/t5-asr-corrector-masked-second-attempt-data/test')

In [6]:
#test_data[0]

In [7]:
test_data.save_to_disk('test')
validation = datasets.Dataset.load_from_disk('test')
validation = validation.train_test_split(0.0625)
validation = validation['test']
validation

Dataset({
    features: ['input', 'output', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 12759
})

In [8]:
train_data.cleanup_cache_files()
validation.cleanup_cache_files()

1

In [9]:
train_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1837181
})

In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [11]:
!mkdir -p t5-asr-corrector-bn

In [12]:
from kaggle_secrets import UserSecretsClient
import wandb
import logging

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api")

wandb.login(key=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [13]:
# defining training related arguments
batch_size = 32
args = TrainingArguments(output_dir="t5-asr-corrector-bn-masked",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=1e-5,
                        num_train_epochs=2,
                        weight_decay=0.01,
                        save_total_limit=2,
                        fp16 = True,
                        gradient_accumulation_steps = 4,
                        eval_steps = 1000,
                        save_steps = 1000,
                        load_best_model_at_end=True,
                        logging_dir="/kaggle/logs",
                        push_to_hub=False,
                       )
                                         

In [14]:
validation

Dataset({
    features: ['input', 'output', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 12759
})

In [15]:
# defining trainer using 🤗
trainer = Trainer(model=model, 
                args=args, 
                train_dataset= train_data,
                eval_dataset=validation,
                tokenizer=tokenizer,
                data_collator=data_collator,
                )

Using cuda_amp half precision backend


In [16]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1837181
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 4
  Total optimization steps = 14352
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: mushrafi88. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
1000,0.174400,0.166853
2000,0.157100,0.149470
3000,0.146900,0.138329
4000,0.139100,0.130829
5000,0.133700,0.125545
6000,0.129700,0.121663
7000,0.126100,0.118613
8000,0.123000,0.116335
9000,0.121500,0.114475
10000,0.119600,0.112959


The following columns in the evaluation set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: input, __index_level_0__, output. If input, __index_level_0__, output are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 12759
  Batch size = 64
Saving model checkpoint to t5-asr-corrector-bn-masked/checkpoint-1000
Configuration saved in t5-asr-corrector-bn-masked/checkpoint-1000/config.json
Model weights saved in t5-asr-corrector-bn-masked/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-asr-corrector-bn-masked/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5-asr-corrector-bn-masked/checkpoint-1000/special_tokens_map.json
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and retu

TrainOutput(global_step=14352, training_loss=0.16206930898370817, metrics={'train_runtime': 35225.606, 'train_samples_per_second': 104.309, 'train_steps_per_second': 0.407, 'total_flos': 3.144680106337567e+17, 'train_loss': 0.16206930898370817, 'epoch': 2.0})

In [17]:
trainer.save_model('t5_asr_bn_corrector_second_attempt')

Saving model checkpoint to t5_asr_bn_corrector_second_attempt
Configuration saved in t5_asr_bn_corrector_second_attempt/config.json
Model weights saved in t5_asr_bn_corrector_second_attempt/pytorch_model.bin
tokenizer config file saved in t5_asr_bn_corrector_second_attempt/tokenizer_config.json
Special tokens file saved in t5_asr_bn_corrector_second_attempt/special_tokens_map.json
